<a href="https://colab.research.google.com/github/JiahuaQu/Colab/blob/main/Clinvar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ref: https://github.com/matthew-j-schmitz/AR-genes-gnomAD/blob/main/parse_for_filter_steps.py

In [28]:
!wget https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh37/clinvar.vcf.gz

--2022-06-29 17:30:35--  https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh37/clinvar.vcf.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 130.14.250.11, 2607:f220:41e:250::13, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58394261 (56M) [application/x-gzip]
Saving to: ‘clinvar.vcf.gz.1’

clinvar.vcf.gz.1    100%[===================>]  55.69M  63.5MB/s    in 0.9s    

2022-06-29 17:30:36 (63.5 MB/s) - ‘clinvar.vcf.gz.1’ saved [58394261/58394261]



In [29]:
!gzip -dk "clinvar.vcf.gz"

gzip: clinvar.vcf already exists; do you wish to overwrite (y or n)? y


In [30]:
import pandas as pd
import numpy as np

In [31]:
clinvar = open('clinvar.vcf', 'r')

In [32]:
list_non_parsed_plp = []
list_of_abbreviated_plp = []
desired_info_fields = ['ALLELEID', 'CLNDISDB', 'CLNHGVS', 'CLNSIG', 'CLNVC', 'MC', 'AF_ESP', 'AF_EXAC', 'AF_TGP', 'RS', 'CLNSIGCONF', 'CLNDISDBINCL', 'GENEINFO']

In [33]:
for line in clinvar:

    plp_non_conf = False # pathogenic or likely pathogenic with no conflicts

    # Remove header lines
    if line.startswith("#"):
        continue

    # Get all fields into separate list item
    split_line = line.split("\t")

    # Remove \n from end of info item
    split_line[-1] = split_line[-1].strip("\n")

    # Split info field into component parts
    split_info_field = split_line[-1].split(";")

    # Remove un-split info field
    split_line.pop()

    # Add each info item to the split line
    for info_item in split_info_field:
        split_line.append(info_item)

    # Find if the line is an autosomal recessive plp with no conflicts
    for field in split_line:
        if field[0:7] == "CLNSIG=":
            if ("athogenic" in field) and ("onflict" not in field):
                plp_non_conf = True

    # If plp with no conflicts, add it to the list
    if plp_non_conf:
        list_non_parsed_plp.append(split_line)

print("Done extracting AR non-conflicting PLP to list_non_parsed_plp")

Done extracting AR non-conflicting PLP to list_non_parsed_plp


In [34]:
# Move through plps in list of plp non conflicting entries
for plp in list_non_parsed_plp:

    # Take CHROM, POS, ID, REF, ALT
    abbreviated_plp = [plp[0], plp[1], plp[2], plp[3], plp[4]]

    # Move through list of desired info fields
    for desired_info_field in desired_info_fields:

        # Initialize variable to track whether the variant contains the field
        field_exists = False

        # Move through the info fields in the current variant
        for existing_info_field in plp:

            # If the current info field is the desired info field
            if existing_info_field.split("=")[0] == desired_info_field:

                # Specify that the field exists
                field_exists = True

                # Add the value of the current info field to the variant
                abbreviated_plp.append(existing_info_field.split("=")[1])

                continue

        # If the desired info field isn't found in the current plp
        if field_exists == False:

            # Add an empty string to the plp
            abbreviated_plp.append("")

    # Exclude X, Y, or MT genes to leave only variants of autosomal genes
    if abbreviated_plp[0] not in ["X", "Y", "MT"]:

        # Append the abbreviated plp variant to the list
        list_of_abbreviated_plp.append(abbreviated_plp)

In [35]:
# Create a dataframe for the abbreviated plp variants
df = pd.DataFrame(list_of_abbreviated_plp, columns = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'ALLELEID', 'CLNDISDB', 'CLNHGVS', 'CLNSIG', 'CLNVC', 'MC', 'AF_ESP', 'AF_EXAC', 'AF_TGP', 'RS', 'CLNSIGCONF', 'CLNDISDBINCL', 'GENEINFO'])

# Convert the dataframe of plp variants to a csv
df.to_csv("all_autosomal_plp_non_conf.csv")

print(df)
print(df.value_counts('CHROM'))

print("Done generating df of all abbreviated plps")

       CHROM       POS       ID REF ALT ALLELEID  \
0          1    879375   950448   C   T   929884   
1          1    899892   916564   C   A   904889   
2          1    949363  1028857   G   A  1015460   
3          1    949523   183381   C   T   181485   
4          1    949696   161455   C  CG   171289   
...      ...       ...      ...  ..  ..      ...   
164411    22  51169571   816970   G   A   806141   
164412    22  51169600  1176013   C   A  1166410   
164413    22  51169630   452885   G   A   446437   
164414    22  51169673   546212   T   C   537027   
164415    22  51169680   985620   C  CA   974264   

                                                 CLNDISDB  \
0                                         MedGen:CN517202   
1       Human_Phenotype_Ontology:HP:0002575,MONDO:MOND...   
2       MONDO:MONDO:0014502,MedGen:C4015293,OMIM:61612...   
3       MONDO:MONDO:0014502,MedGen:C4015293,OMIM:61612...   
4       MONDO:MONDO:0014502,MedGen:C4015293,OMIM:61612...   
...      

In [36]:
clinvar.close()